# Eclipse Thermal Response with heat1d

This notebook demonstrates how to:
1. Generate an absorbed solar flux file with an eclipse event
2. Run the heat1d thermal model for three thermophysical cases (standard, $2\times$ lower, $10\times$ lower surface conductivity and density)
3. Plot the full diurnal temperature curves alongside a zoom on the eclipse

**Local time convention:** 0 = local noon (hour angle = 0). One "local hour" = `planet.day / 24` in SI seconds. On the Moon, 1 local hour $\approx$ 29.5 Earth hours.

In [ ]:
# ---- User parameters ----
latitude_deg = 0.0           # Latitude [degrees]
eclipse_start_hr = 1.0       # Eclipse start [local hours past noon]
eclipse_duration_s = 7200.0  # Eclipse duration [SI seconds]
eclipse_fraction = 1.0       # Fraction of flux blocked (1.0 = total)

# Thermophysical cases: (label, ks_factor, rhos_factor)
# Factors are applied relative to the standard Moon values
# (ks=7.4e-4 W/m/K, rhos=1100 kg/m^3)
cases = [
    ("Standard",          1.0,  1.0),
    (r"$k_s, \rho_s$ / 2",  0.5,  0.5),
    (r"$k_s, \rho_s$ / 10", 0.1,  0.1),
]

In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as plt
import planets

from heat1d.generate_flux import compute_flux_array, apply_eclipse, _required_nsteps
from heat1d.flux import write_flux_file, read_flux_file
from heat1d.config import Configurator
from heat1d.model import Model

## 1. Generate the flux file

The flux depends only on orbital geometry and albedo, not on thermophysical properties, so we compute it once.

In [ ]:
planet = planets.Moon
lat_rad = np.deg2rad(latitude_deg)

# Choose enough time steps to resolve the eclipse (>= 10 samples)
nsteps_eclipse = _required_nsteps(planet, 0.0, 24.0, eclipse_duration_s)
nsteps = max(480, nsteps_eclipse)

# Eclipse flux
flux_ecl, dt = compute_flux_array(planet, lat_rad, nsteps)
n_affected, n_daytime = apply_eclipse(
    flux_ecl, dt, planet, 0.0,
    eclipse_start_hr, eclipse_duration_s, eclipse_fraction,
)
flux_path = "eclipse_flux.txt"
write_flux_file(flux_path, flux_ecl, dt)

# Reference flux (no eclipse)
flux_ref, _ = compute_flux_array(planet, lat_rad, nsteps)
flux_ref_path = "reference_flux.txt"
write_flux_file(flux_ref_path, flux_ref, dt)

print(f"Flux: {nsteps} samples, dt = {dt:.1f} s ({dt/3600:.2f} Earth hr)")
print(f"Eclipse: {n_affected} samples affected, {n_daytime} during daytime")

## 2. Run heat1d for each thermophysical case

Each case modifies the surface conductivity $k_s$ and surface density $\rho_s$ relative to the standard Moon values. Lower values correspond to lower thermal inertia, which produces faster and deeper cooling during the eclipse.

In [ ]:
output_interval_s = dt  # match flux file resolution

# Store results: list of (label, eclipse_model, reference_model)
results = []

for label, ks_fac, rhos_fac in cases:
    # Create a modified planet for this case
    p = copy.copy(planet)
    p.ks = planet.ks * ks_fac
    p.rhos = planet.rhos * rhos_fac
    
    print(f"--- {label}: ks={p.ks:.2e} W/m/K, rhos={p.rhos:.0f} kg/m^3 ---")
    
    # Eclipse run
    config_ecl = Configurator(
        solver="implicit",
        output_interval=output_interval_s,
        adaptive_tol=None,
    )
    flux_series, flux_dt = read_flux_file(flux_path)
    model_ecl = Model(
        planet=p, lat=lat_rad, ndays=1, config=config_ecl,
        flux_series=flux_series, flux_dt=flux_dt,
    )
    model_ecl.run()
    
    # Reference run (no eclipse)
    config_ref = Configurator(
        solver="implicit",
        output_interval=output_interval_s,
        adaptive_tol=None,
    )
    flux_series_ref, flux_dt_ref = read_flux_file(flux_ref_path)
    model_ref = Model(
        planet=p, lat=lat_rad, ndays=1, config=config_ref,
        flux_series=flux_series_ref, flux_dt=flux_dt_ref,
    )
    model_ref.run()
    
    results.append((label, model_ecl, model_ref))
    print(f"  T_max={model_ecl.T[:,0].max():.1f} K, "
          f"T_min={model_ecl.T[:,0].min():.1f} K")

print("\nAll runs complete.")

## 3. Plot results

In [ ]:
colors = ['C0', 'C1', 'C3']  # blue, orange, red

ecl_start = eclipse_start_hr
ecl_end = ecl_start + eclipse_duration_s / planet.day * 24.0
ecl_margin_hr = max(0.5, (ecl_end - ecl_start) * 3)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

for i, (label, model_ecl, model_ref) in enumerate(results):
    c = colors[i]
    # Left panel: full diurnal cycle
    ax1.plot(model_ref.lt, model_ref.T[:, 0], '--', color=c, lw=0.8, alpha=0.4)
    ax1.plot(model_ecl.lt, model_ecl.T[:, 0], '-', color=c, lw=1.2, label=label)
    
    # Right panel: zoom on eclipse
    ax2.plot(model_ref.lt, model_ref.T[:, 0], '--', color=c, lw=1, alpha=0.4)
    ax2.plot(model_ecl.lt, model_ecl.T[:, 0], '-', color=c, lw=1.5, label=label)

# Eclipse shading
for ax in (ax1, ax2):
    ax.axvspan(ecl_start, ecl_end, alpha=0.12, color='gray')

# Left panel formatting
ax1.set_xlabel('Local time [hr past noon]')
ax1.set_ylabel('Surface temperature [K]')
ax1.set_title(f'Full diurnal cycle — lat = {latitude_deg:.1f}°')
ax1.set_xlim(0, 24)
ax1.legend(fontsize=9)

# Right panel formatting
zoom_start = max(0, ecl_start - ecl_margin_hr)
zoom_end = min(24, ecl_end + ecl_margin_hr)
ax2.set_xlim(zoom_start, zoom_end)
ax2.set_xlabel('Local time [hr past noon]')
ax2.set_ylabel('Surface temperature [K]')
ecl_dur_min = eclipse_duration_s / 60
ax2.set_title(f'Eclipse detail — {ecl_dur_min:.0f} min ({eclipse_duration_s:.0f} s)')

# Annotate peak cooling for each case
mask = (results[0][1].lt >= ecl_start) & (results[0][1].lt <= ecl_end + ecl_margin_hr / 2)
for i, (label, model_ecl, model_ref) in enumerate(results):
    if mask.any():
        dT = model_ecl.T[mask, 0] - model_ref.T[mask, 0]
        idx_min = np.argmin(dT)
        y_offset = -20 - 15 * i  # stagger annotations
        ax2.annotate(
            f'$\\Delta T = {dT[idx_min]:.1f}$ K',
            xy=(model_ecl.lt[mask][idx_min], model_ecl.T[mask, 0][idx_min]),
            xytext=(15, y_offset), textcoords='offset points',
            arrowprops=dict(arrowstyle='->', color=colors[i]),
            fontsize=9, color=colors[i],
        )

ax2.legend(fontsize=9)
fig.tight_layout()
plt.show()

## Summary table

In [ ]:
mask = (results[0][1].lt >= ecl_start) & (results[0][1].lt <= ecl_end + ecl_margin_hr / 2)

print(f"{'Case':<20s}  {'ks [W/m/K]':>12s}  {'rhos [kg/m3]':>12s}  "
      f"{'Peak dT [K]':>12s}  {'T at peak [K]':>13s}")
print("-" * 78)

for i, (label_raw, model_ecl, model_ref) in enumerate(results):
    _, ks_fac, rhos_fac = cases[i]
    ks_val = planet.ks * ks_fac
    rhos_val = planet.rhos * rhos_fac
    dT = model_ecl.T[mask, 0] - model_ref.T[mask, 0]
    idx_min = np.argmin(dT)
    # Strip LaTeX for the table
    name = label_raw.replace(r'$k_s, \rho_s$', 'ks,rhos')
    print(f"{name:<20s}  {ks_val:>12.2e}  {rhos_val:>12.0f}  "
          f"{dT[idx_min]:>12.1f}  {model_ecl.T[mask, 0][idx_min]:>13.1f}")